# 한국어 텍스트 감정 분석 - 데이터 전처리

이 노트북은 한국어 영화 리뷰 감정 분석을 위한 데이터 전처리 및 탐색적 데이터 분석(EDA)을 수행합니다.

## 주요 기능
- 데이터 품질 검사 및 정제
- 텍스트 전처리 파이프라인
- 훈련/검증 데이터 분할
- 전처리된 데이터 CSV 저장


### IMPORT

In [1]:
# 한국어 텍스트 감정 분석을 위한 필수 라이브러리들
from collections import Counter  # 카운터 자료구조
import os  # 운영체제 인터페이스
import platform  # 플랫폼 정보
import re  # 정규 표현식
import sys  # 시스템 정보
import warnings  # 경고 메시지 제어
import math


import matplotlib.pyplot as plt  # 데이터 시각화
plt.rc("font", family="NanumBarunGothic")  # 한글 폰트 설정(없으면 설치 필요)

import numpy as np  # 수치 연산
import pandas as pd  # 데이터 처리 및 분석
import seaborn as sns  # 고급 시각화
import torch  # 딥러닝 프레임워크
import koreanize_matplotlib
import wandb
# 머신러닝 관련 라이브러리
from sklearn.metrics import accuracy_score, f1_score  # 평가 지표
from sklearn.model_selection import train_test_split # 데이터 분할

# 트랜스포머 및 BERT 관련 라이브러리
from transformers import (
    AutoModelForSequenceClassification,  # 시퀀스 분류 모델
    AutoTokenizer,  # 토크나이저
    DataCollatorWithPadding,  # 패딩 데이터 콜레이터
    set_seed,
    Trainer,  # 트레이너
    TrainingArguments,  # 훈련 설정
)

# PyTorch 데이터 처리
from torch.utils.data import Dataset  # 데이터셋 및 데이터로더

# 경고 메시지 필터링
warnings.filterwarnings("ignore")

# 라이브러리 버전 정보 출력 (재현성을 위함)
print("=== 라이브러리 버전 정보 ===")
print(f"Python: {sys.version}")
print(f"Platform: {platform.platform()}")
print(f"pandas: {pd.__version__}")
print(f"numpy: {np.__version__}")
print(f"torch: {torch.__version__}")
print(f"transformers: {__import__('transformers').__version__}")
print(f"sklearn: {__import__('sklearn').__version__}")
print(f"matplotlib: {__import__('matplotlib').__version__}")
print(f"seaborn: {sns.__version__}")

# GPU 사용 가능 여부 확인
print("\n=== PyTorch GPU 지원 정보 ===")
print(f"CUDA 사용 가능: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA 버전: {torch.version.cuda}")
    print(f"GPU 개수: {torch.cuda.device_count()}")
    print(f"현재 GPU: {torch.cuda.current_device()}")
    print(f"GPU 이름: {torch.cuda.get_device_name()}")
else:
    print("CPU에서 실행 중")


/data/ephemeral/home/py310/lib/python3.10/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/data/ephemeral/home/py310/lib/python3.10/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotat

=== 라이브러리 버전 정보 ===
Python: 3.10.13 (main, Sep 11 2023, 13:44:35) [GCC 11.2.0]
Platform: Linux-5.4.0-99-generic-x86_64-with-glibc2.31
pandas: 2.2.3
numpy: 1.26.4
torch: 2.6.0+cu124
transformers: 4.55.0
sklearn: 1.6.1
matplotlib: 3.10.5
seaborn: 0.13.2

=== PyTorch GPU 지원 정보 ===
CUDA 사용 가능: True
CUDA 버전: 12.4
GPU 개수: 1
현재 GPU: 0
GPU 이름: Tesla V100-SXM2-32GB


In [2]:
from dotenv import load_dotenv
import os

# 이 함수가 .env 파일을 읽어서 환경 변수로 로드합니다.
load_dotenv()


True

## Random Seed Configuration


In [3]:
# 랜덤 시드 설정
RANDOM_STATE = 42


set_seed(RANDOM_STATE)

print(f"랜덤 시드 {RANDOM_STATE}로 설정 완료")


랜덤 시드 42로 설정 완료


# Data Load


In [4]:
# 데이터 로드
df = pd.read_csv("data/train_newly_gen_added.csv")

# 처음 몇 행 표시
print("\n처음 5행:")
df.head()



처음 5행:


,ID,review,label,type
0,0,이 영화는 정말 여성의 강인함과 힘을 제대로 보여주는 작품이었어요! 주인공이 자기 ...,2,augment
1,1,어느 부잣집 도련님의 철없는 행각,1,original
2,3,왜이렇게 재미가없냐 원도 별로였지만 원보다 더 재미없네,0,original
3,4,크리스마스 시즌엔 무조건 홈 알론이죠! 맥컬리 컬킨이 연기한 케빈의 재치있고 천방지...,2,augment
4,5,참나ㅋㅋ이게 무슨 드라마 최초 뮤지컬드라마야 이게무슨ㅋㅋ걍 다른 드라마랑 똑같구만 ...,0,original


# Data/Feature Engineering


In [5]:
# 데이터 전처리를 위한 복사본 생성
df_processed = df.copy()

print(f"원본 데이터셋 크기: {len(df_processed):,}개")

df_processed.tail()


원본 데이터셋 크기: 374,644개


,ID,review,label,type
374639,374639,"포스터만 그럴싸하고 13구역이라는 이름만 빌려 쓴 영화다. 완전 졸작이고, 액션 신...",0,newly_generated
374640,374640,요지는 이해되지만 생활 패턴을 유지하고 식단만 바꾸는 게 과연 올바른 실험 방법일까...,1,newly_generated
374641,374641,"말이 필요 없다, 으리 하나만 기억하면 충분하다",3,newly_generated
374642,374642,솔직히 이건 정말 재미가 없고 한숨만 나온다.,0,newly_generated
374643,374643,정말 훌륭한 영화였어요.,2,newly_generated


In [6]:
def cutting_augmented_data(df, text_col='review', label_col='label', id_col='ID'):
    """
    데이터 증강을 위한 데이터 커팅 함수
    augmented 데이터의 리뷰를 구두점 기준으로 나누어 새로운 데이터 생성
    - 2-5개 문장: 2등분
    - 6개 이상 문장: 3등분
    
    Args:
        df: 원본 데이터프레임
        text_col: 텍스트 컬럼명
        label_col: 라벨 컬럼명  
        id_col: ID 컬럼명
    
    Returns:
        new_df: 분할된 데이터가 추가된 새로운 데이터프레임
    """
    import re
    
    # augmented 데이터만 필터링
    df_augmented = df[df["type"] == "augment"].copy()
    df_original = df[df["type"] == "original"].copy()
    df_newly_gen = df[df["type"] == "newly_generated"].copy()
    
    print(f"원본 augmented 데이터: {len(df_augmented)}개")
    
    new_data = []
    
    for idx, row in df_augmented.iterrows():
        text = row[text_col]
        label = row[label_col]
        original_id = row[id_col]
        
        # 구두점을 기준으로 문장 분할
        # 마침표, 느낌표, 물음표를 기준으로 분할
        sentences = re.split(r'[.!?]+', text)
        
        # 빈 문장 제거 및 공백 정리
        sentences = [s.strip() for s in sentences if s.strip()]
        
        if len(sentences) >= 2:  # 2개 이상의 문장이 있을 때만 분할
            if len(sentences) >= 6:  # 6개 이상 문장이면 3등분
                # 3등분
                part_size = len(sentences) // 3
                
                # 첫 번째 부분
                first_part = '. '.join(sentences[:part_size])
                if first_part.strip():
                    new_data.append({
                        id_col: f"{original_id}_part1",
                        text_col: first_part.strip(),
                        label_col: label,
                        "type": "augment"
                    })
                
                # 두 번째 부분
                second_part = '. '.join(sentences[part_size:part_size*2])
                if second_part.strip():
                    new_data.append({
                        id_col: f"{original_id}_part2", 
                        text_col: second_part.strip(),
                        label_col: label,
                        "type": "augment"
                    })
                
                # 세 번째 부분
                third_part = '. '.join(sentences[part_size*2:])
                if third_part.strip():
                    new_data.append({
                        id_col: f"{original_id}_part3",
                        text_col: third_part.strip(),
                        label_col: label,
                        "type": "augment"
                    })
                    
            else:  # 2-5개 문장이면 2등분
                # 대략 절반으로 나누기
                mid_point = len(sentences) // 2
                
                # 첫 번째 절반
                first_half = '. '.join(sentences[:mid_point])
                if first_half.strip():
                    new_data.append({
                        id_col: f"{original_id}_part1",
                        text_col: first_half.strip(),
                        label_col: label,
                        "type": "augment"
                    })
                
                # 두 번째 절반  
                second_half = '. '.join(sentences[mid_point:])
                if second_half.strip():
                    new_data.append({
                        id_col: f"{original_id}_part2", 
                        text_col: second_half.strip(),
                        label_col: label,
                        "type": "augment"
                    })
        else:
            # 분할할 수 없는 경우 원본 유지
            new_data.append({
                id_col: original_id,
                text_col: text,
                label_col: label,
                "type": "augment"
            })
    
    # 새로운 데이터프레임 생성
    new_df = pd.DataFrame(new_data)
    
    # 원본 데이터와 합치기
    result_df = pd.concat([df_original, new_df, df_newly_gen], ignore_index=True)
    
    print(f"분할 후 새로운 데이터: {len(new_df)}개")
    print(f"최종 데이터: {len(result_df)}개")
    
    return result_df

In [7]:
def clean_data_quality(df, text_col='review', label_col='label', id_col='ID'):
    """
    데이터 품질 검사 및 문제가 있는 데이터 제거
    
    Args:
        df: 원본 데이터프레임
        text_col: 텍스트 컬럼명
        label_col: 라벨 컬럼명  
        id_col: ID 컬럼명
    
    Returns:
        clean_df: 품질 문제가 제거된 데이터프레임
        removed_info: 제거된 데이터 정보
    """
    print(f"원본 데이터 수: {len(df):,}개")
    
    # 1. 품질 검사
    null_mask = df[text_col].isnull()
    empty_mask = df[text_col].str.strip().eq("")
    whitespace_mask = df[text_col].str.isspace()
    digit_only_mask = df[text_col].str.match(r"^\d+$", na=False)
    
    # 중복 리뷰 처리 - keep='first'로 첫 번째만 유지
    duplicate_mask = df[text_col].duplicated(keep='first')
    
    print(f"Null 값: {null_mask.sum()}개")
    print(f"빈 텍스트: {empty_mask.sum()}개") 
    print(f"공백만 있는 텍스트: {whitespace_mask.sum()}개")
    print(f"중복 리뷰 (제거될 개수): {duplicate_mask.sum()}개")
    print(f"숫자만 있는 리뷰: {digit_only_mask.sum()}개")
    
    # 2. 품질 문제가 있는 데이터 마스크 생성
    quality_issues_mask = (
        null_mask | 
        empty_mask | 
        whitespace_mask | 
        duplicate_mask | 
        digit_only_mask 
    )
    
    # 3. 품질 문제가 있는 데이터 제거
    clean_df = df[~quality_issues_mask].copy().reset_index(drop=True)
    removed_count = quality_issues_mask.sum()
    
    print(f"\n=== 데이터 품질 검사 결과 ===")
    print(f"제거된 데이터: {removed_count:,}개")
    print(f"남은 데이터: {len(clean_df):,}개")
    print(f"데이터 품질 비율: {len(clean_df) / len(df) * 100:.2f}%")
    
    # 4. 제거된 데이터의 상세 정보 출력
    if removed_count > 0:
        print(f"\n=== 제거된 데이터 상세 정보 ===")
        removed_data = df[quality_issues_mask]
        
        if null_mask.sum() > 0:
            print(f"Null 값 샘플: {removed_data[null_mask][text_col].head(3).tolist()}")
        if empty_mask.sum() > 0:
            print(f"빈 텍스트 샘플: {removed_data[empty_mask][text_col].head(3).tolist()}")
        if whitespace_mask.sum() > 0:
            print(f"공백만 있는 텍스트 샘플: {removed_data[whitespace_mask][text_col].head(3).tolist()}")
        if duplicate_mask.sum() > 0:
            print(f"중복 리뷰 샘플 (제거된 것들): {removed_data[duplicate_mask][text_col].head(3).tolist()}")
        if digit_only_mask.sum() > 0:
            print(f"숫자만 있는 리뷰 샘플: {removed_data[digit_only_mask][text_col].head(3).tolist()}")
    
    # 5. 제거된 데이터 정보 저장
    removed_info = {
        'total_removed': removed_count,
        'null_count': null_mask.sum(),
        'empty_count': empty_mask.sum(),
        'whitespace_count': whitespace_mask.sum(),
        'duplicate_count': duplicate_mask.sum(),
        'digit_only_count': digit_only_mask.sum()
    }
    
    return clean_df, removed_info

print("✅ 데이터 품질 검사 및 제거 함수 정의 완료")

✅ 데이터 품질 검사 및 제거 함수 정의 완료


In [8]:
# 텍스트 전처리 파이프라인 클래스 구성
class TextPreprocessingPipeline:
    """
    텍스트 전처리 파이프라인 클래스
    - 기본 전처리와 학습 데이터 기반 고급 전처리를 통합 관리
    - 재사용 가능하고 확장 가능한 구조
    """

    def __init__(self):
        self.is_fitted = False
        self.vocab_info = {}
        self.label_patterns = {}

    def basic_preprocess(self, texts):
        """기본 전처리 (clean_text + normalize 기능)"""
        processed_texts = []
        for text in texts:
            # 기본 텍스트 정리
            cleaned = self._clean_text(text)
            processed_texts.append(cleaned)
        return processed_texts

    def _clean_text(self, text):
        """기존 clean_text 함수 내용"""
        if pd.isna(text):
            return ""

        text = str(text).strip()
        text = text.lower() # 소문자 변환
        text = self._remove_urls_emails_mentions(text) # URL, 이메일, 멘션 제거
        text = self._normalize_punctuation(text)  # 구두점 정규화
        #text = self._remove_incomplete_korean(text)
        text = self._normalize_emotion_expressions(text) # 감정 표현 정규화 (ㅋㅋㅋ , ㅎㅎㅎ)
        text = self._reduce_excessive_repetition(text) # 과도한 문자 반복 축소 (아아아아아아앙 -> 아아아아)
        text = self._clean_special_characters(text) # 특수문자 제거 (이모티콘, 특수기호)
        text = self._normalize_whitespace(text) # 공백 정규화 (여러 개의 공백 -> 하나의 공백)

        return text.strip()

    def fit(self, texts, labels=None):
        """학습 데이터로부터 전처리 정보 학습 (품질 검사 기준 학습)"""

        self.is_fitted = True
        print("✓ 전처리 파이프라인 학습 완료")


    def transform(self, texts):
        """전처리 적용 (품질 문제 데이터 제거 + 텍스트 전처리)"""
        if not self.is_fitted:
            print(
                "Warning: 파이프라인이 학습되지 않았습니다. 기본 전처리만 적용합니다."
            )
            return self.basic_preprocess(texts)
        
        # 텍스트 전처리 적용
        return self.basic_preprocess(texts)

    def fit_transform(self, texts, labels=None):
        """학습과 변환을 동시에 수행"""
        # 1. 학습 단계 (품질 검사 기준 학습)
        self.fit(texts, labels)
        
        # 2. 변환 단계 (품질 문제 데이터 제거 + 텍스트 전처리)
        processed_texts = self.transform(texts)
        
        # 3. 라벨도 동일하게 필터링
        return processed_texts


    @staticmethod
    def _remove_incomplete_korean(text):
        """불완전한 한글 제거 (자음/모음만 있는 경우)"""
        return re.sub(r"[ㄱ-ㅎㅏ-ㅣ]+", "", text)

    @staticmethod
    def _normalize_emotion_expressions(text):
        """감정 표현 정규화"""
        def replace_emotion(match):
            char = match.group(1)
            count = len(match.group(0))
            # log2x + 1 공식을 정수로 변환
            new_count = int(math.log2(count)) + 1 if count > 0 else 1
            return char * new_count
        
        # 웃음과 슬픔 표현 정규화 (2번 이상 반복)
        text = re.sub(r"([ㅋㅎ])\1+", replace_emotion, text)
        text = re.sub(r"([ㅠㅜㅡ])\1+", replace_emotion, text)
        return text

    @staticmethod
    def _reduce_excessive_repetition(text):
        """과도한 문자 반복 축소 (4번 이상 → 3번으로)"""
        
        def replace_repetition(match):
            char = match.group(1)
            count = len(match.group(0))
            # log2x + 1 공식을 정수로 변환하고 최소 1개 보장
            new_count = max(1, int(math.log2(count)) + 1) if count > 0 else 1
            return char * new_count
        
        return re.sub(r"(.)\1{3,}", replace_repetition, text)

    @staticmethod
    def _clean_special_characters(text):
        """특수문자 제거 (이모티콘 보존)"""

        # 1. 허용할 이모티콘 범위 정의
        # emoji_ranges = r"\U0001F600-\U0001F64F"  # Emoticons
        # emoji_ranges += r"\U0001F300-\U0001F5FF"  # Misc Symbols/Pictographs
        # emoji_ranges += r"\U0001F680-\U0001F6FF"  # Transport/Map
        # emoji_ranges += r"\U00002600-\U000026FF"  # Misc Symbols (★ 포함)
        # emoji_ranges += r"\U00002700-\U000027BF"  # Dingbats

        # 2. 허용할 기타 특수기호 정의
        other_symbols = r"@★#$" # 예시로 @ 추가

        # 3. 허용할 문자들을 조합하여 정규식 생성
        #allowed_chars = rf"\w\s가-힣.,!?ㅋㅎㅠㅜㅡ~\-{emoji_ranges}{other_symbols}"
        allowed_chars = rf"\w\s가-힣.,!?ㅋㅎㅠㅜㅡ~\-"
        
        return re.sub(rf"[^{allowed_chars}]", " ", text)
 
 
    @staticmethod
    def _normalize_whitespace(text):
        """공백 정규화"""
        return re.sub(r"\s+", " ", text)

    @staticmethod
    def _normalize_punctuation(text):
        """구두점 정규화 (log 공식 적용)"""
        def replace_punctuation(match):
            char = match.group(1)
            count = len(match.group(0))
            # log2x + 1 공식을 정수로 변환
            new_count = int(math.log2(count)) + 1 if count > 0 else 1
            return char * new_count
        
        # 각 구두점별로 log 공식 적용
        text = re.sub(r"([.])\1+", replace_punctuation, text)
        text = re.sub(r"([!])\1+", replace_punctuation, text)
        text = re.sub(r"([?])\1+", replace_punctuation, text)
        text = re.sub(r"([,])\1+", replace_punctuation, text)
        
        # 구두점 앞뒤 공백 정리
        text = re.sub(r"\s+([.,!?])", r"\1", text)
        text = re.sub(r"([.,!?])\s+", r"\1 ", text)
        
        return text

    @staticmethod
    def _remove_urls_emails_mentions(text):
        """URL, 이메일, 멘션 제거"""
        # URL 패턴 제거
        text = re.sub(
            r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+",
            "",
            text,
        )
        # 이메일 패턴 제거
        text = re.sub(r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}\b", "", text)
        # 멘션 패턴 제거
        text = re.sub(r"\B@\w+", "", text)
        
        return text


In [9]:
preprocessor = TextPreprocessingPipeline()


## 데이터 분할 전략

- 훈련/검증 데이터 분할
- 클래스 분포를 유지하는 계층적 분할


In [10]:
# 데이터 품질 검사 및 제거
#df_processed = cutting_augmented_data(df_processed)
df_clean, removed_info = clean_data_quality(df_processed)

print(f"\n원본 데이터: {len(df_processed):,}개")
print(f"정제된 데이터: {len(df_clean):,}개")
print(f"제거된 데이터: {removed_info['total_removed']:,}개")


원본 데이터 수: 374,644개
Null 값: 1715개
빈 텍스트: 0개
공백만 있는 텍스트: 0개
중복 리뷰 (제거될 개수): 7672개
숫자만 있는 리뷰: 64개

=== 데이터 품질 검사 결과 ===
제거된 데이터: 7,701개
남은 데이터: 366,943개
데이터 품질 비율: 97.94%

=== 제거된 데이터 상세 정보 ===
Null 값 샘플: [nan, nan, nan]
중복 리뷰 샘플 (제거된 것들): ['재미없다', '굳', 'tv 전기세가 아깝다!!!']
숫자만 있는 리뷰 샘플: ['1234567890', '2', '1']

원본 데이터: 374,644개
정제된 데이터: 366,943개
제거된 데이터: 7,701개


In [11]:
df_processed

,ID,review,label,type
0,0,이 영화는 정말 여성의 강인함과 힘을 제대로 보여주는 작품이었어요! 주인공이 자기 ...,2,augment
1,1,어느 부잣집 도련님의 철없는 행각,1,original
2,3,왜이렇게 재미가없냐 원도 별로였지만 원보다 더 재미없네,0,original
3,4,크리스마스 시즌엔 무조건 홈 알론이죠! 맥컬리 컬킨이 연기한 케빈의 재치있고 천방지...,2,augment
4,5,참나ㅋㅋ이게 무슨 드라마 최초 뮤지컬드라마야 이게무슨ㅋㅋ걍 다른 드라마랑 똑같구만 ...,0,original
...,...,...,...,...
374639,374639,"포스터만 그럴싸하고 13구역이라는 이름만 빌려 쓴 영화다. 완전 졸작이고, 액션 신...",0,newly_generated
374640,374640,요지는 이해되지만 생활 패턴을 유지하고 식단만 바꾸는 게 과연 올바른 실험 방법일까...,1,newly_generated
374641,374641,"말이 필요 없다, 으리 하나만 기억하면 충분하다",3,newly_generated
374642,374642,솔직히 이건 정말 재미가 없고 한숨만 나온다.,0,newly_generated


In [12]:
LABEL_MAPPING = {0: "강한 부정", 1: "약한 부정", 2: "약한 긍정", 3: "강한 긍정"}

In [13]:
# 타입별 상세 분석
df = df_clean

print("=" * 50)
print("타입별 데이터 분석")
print("=" * 50)

# 타입별 기본 통계
for data_type in df["type"].unique():
    type_data = df[df["type"] == data_type]
    print(f"\n[{data_type.upper()} 데이터]")
    print(f"  총 개수: {len(type_data):,}개")
    print(f"  비율: {len(type_data)/len(df)*100:.1f}%")
    
    # 타입별 클래스 분포
    print(f"  클래스 분포:")
    type_class_counts = type_data["label"].value_counts().sort_index()
    type_class_percentages = type_data["label"].value_counts(normalize=True).sort_index() * 100
    for label, count in type_class_counts.items():
        percentage = type_class_percentages[label]
        print(f"    클래스 {label} ({LABEL_MAPPING[label]}): {count:,}개 ({percentage:.1f}%)")
    
    # 타입별 텍스트 길이 통계
    type_data["text_length"] = type_data["review"].str.len()
    type_data["word_count"] = type_data["review"].str.split().str.len()
    
    print(f"  텍스트 길이 통계:")
    print(f"    평균 문자 수: {type_data['text_length'].mean():.1f}")
    print(f"    평균 단어 수: {type_data['word_count'].mean():.1f}")
    print(f"    최대 문자 수: {type_data['text_length'].max()}")
    print(f"    최소 문자 수: {type_data['text_length'].min()}")

print("\n✅ 타입별 분석 완료")

타입별 데이터 분석

[AUGMENT 데이터]
  총 개수: 139,818개
  비율: 38.1%
  클래스 분포:
    클래스 0 (강한 부정): 57,029개 (40.8%)
    클래스 1 (약한 부정): 13,607개 (9.7%)
    클래스 2 (약한 긍정): 49,706개 (35.6%)
    클래스 3 (강한 긍정): 19,476개 (13.9%)
  텍스트 길이 통계:
    평균 문자 수: 282.0
    평균 단어 수: 69.9
    최대 문자 수: 2035
    최소 문자 수: 44

[ORIGINAL 데이터]
  총 개수: 136,453개
  비율: 37.2%
  클래스 분포:
    클래스 0 (강한 부정): 55,837개 (40.9%)
    클래스 1 (약한 부정): 13,310개 (9.8%)
    클래스 2 (약한 긍정): 48,188개 (35.3%)
    클래스 3 (강한 긍정): 19,118개 (14.0%)
  텍스트 길이 통계:
    평균 문자 수: 36.0
    평균 단어 수: 7.8
    최대 문자 수: 142
    최소 문자 수: 1

[NEWLY_GENERATED 데이터]
  총 개수: 90,672개
  비율: 24.7%
  클래스 분포:
    클래스 0 (강한 부정): 22,598개 (24.9%)
    클래스 1 (약한 부정): 22,806개 (25.2%)
    클래스 2 (약한 긍정): 22,501개 (24.8%)
    클래스 3 (강한 긍정): 22,767개 (25.1%)
  텍스트 길이 통계:
    평균 문자 수: 44.1
    평균 단어 수: 10.9
    최대 문자 수: 329
    최소 문자 수: 4

✅ 타입별 분석 완료


In [14]:
# 데이터 분할 - validation은 original만, train은 모든 타입 포함
print("데이터 분할 전략:")
print("- Train: original + augment + newly_generated (모든 타입)")
print("- Validation: original만 (원본 데이터만으로 검증)")

# 1. Original 데이터만 추출 (validation용)
original_mask = df_clean["type"] == "original"
df_original = df_clean[original_mask].copy()

# 2. Original 데이터를 train/val로 분할 (validation은 original의 10%)
X_original = df_original["review"]
y_original = df_original["label"]
ids_original = df_original["ID"]
types_original = df_original["type"]

# Original 데이터를 90:10으로 분할 (validation은 original의 10%)
X_original_train, X_original_val, y_original_train, y_original_val, ids_original_train, ids_original_val, types_original_train, types_original_val = train_test_split(
    X_original, y_original, ids_original, types_original, 
    test_size=0.05, random_state=RANDOM_STATE, stratify=y_original
)

# 3. Non-original 데이터 (augment + newly_generated) - 모두 train에 포함
non_original_mask = df_clean["type"] != "original"
df_non_original = df_clean[non_original_mask].copy()

# 4. 최종 train 데이터: original_train + non_original (모든 타입)
X_train = pd.concat([X_original_train, df_non_original["review"]], ignore_index=True)
y_train = pd.concat([y_original_train, df_non_original["label"]], ignore_index=True)
ids_train = pd.concat([ids_original_train, df_non_original["ID"]], ignore_index=True)
types_train = pd.concat([types_original_train, df_non_original["type"]], ignore_index=True)

# 5. 최종 validation 데이터: original_val만
X_val = X_original_val
y_val = y_original_val
ids_val = ids_original_val
types_val = types_original_val

print(f"\n전체 데이터: {len(df_clean):,}개")
print(f"  - Original: {len(df_original):,}개")
print(f"  - Augment + Newly_generated: {len(df_non_original):,}개")
print(f"훈련 데이터: {len(X_train):,}개")
print(f"검증 데이터: {len(X_val):,}개")

# 텍스트 전처리 파이프라인 적용
print("훈련 데이터에 대한 전처리 파이프라인 학습 및 적용...")
X_train_processed = preprocessor.fit_transform(X_train.tolist(), y_train.tolist())

print("검증 데이터에 전처리 파이프라인 적용...")
X_val_processed = preprocessor.transform(X_val.tolist())

# 원본 데이터프레임 구조로 분할된 데이터 생성 - 모델 학습용 형태 (type 컬럼 포함)
train_data = pd.DataFrame(
    {"ID": ids_train, "review": X_train_processed, "label": y_train, "type": types_train}
).reset_index(drop=True)

val_data = pd.DataFrame(
    {"ID": ids_val, "review": X_val_processed, "label": y_val, "type": types_val}
).reset_index(drop=True)

print(f"\nTrain: {len(train_data)}, Val: {len(val_data)}")

# 계층 분할이 올바르게 수행되었는지 검증 - 클래스 분포 확인
print("\n클래스 분포 검증:")
print("전체 데이터:")
total_distribution = df_clean["label"].value_counts(normalize=True).sort_index()
total_counts = df_clean["label"].value_counts().sort_index()
for idx, val in total_distribution.items():
    count = total_counts[idx]
    print(f"  클래스 {idx}: {count}개 ({val * 100:.1f}%)")

print("\nOriginal 데이터:")
original_distribution = df_original["label"].value_counts(normalize=True).sort_index()
original_counts = df_original["label"].value_counts().sort_index()
for idx, val in original_distribution.items():
    count = original_counts[idx]
    print(f"  클래스 {idx}: {count}개 ({val * 100:.1f}%)")

print("\n훈련 데이터:")
train_distribution = train_data["label"].value_counts(normalize=True).sort_index()
train_counts = train_data["label"].value_counts().sort_index()
for idx, val in train_distribution.items():
    count = train_counts[idx]
    print(f"  클래스 {idx}: {count}개 ({val * 100:.1f}%)")

print("\n검증 데이터:")
val_distribution = val_data["label"].value_counts(normalize=True).sort_index()
val_counts = val_data["label"].value_counts().sort_index()
for idx, val in val_distribution.items():
    count = val_counts[idx]
    print(f"  클래스 {idx}: {count}개 ({val * 100:.1f}%)")

# 타입별 분포 확인
print("\n타입별 분포:")
print("훈련 데이터 타입 분포:")
train_type_counts = train_data["type"].value_counts()
for type_name, count in train_type_counts.items():
    percentage = count / len(train_data) * 100
    print(f"  {type_name}: {count:,}개 ({percentage:.1f}%)")

print("\n검증 데이터 타입 분포:")
val_type_counts = val_data["type"].value_counts()
for type_name, count in val_type_counts.items():
    percentage = count / len(val_data) * 100
    print(f"  {type_name}: {count:,}개 ({percentage:.1f}%)")

# 전처리 결과 확인
print("\n전처리 결과 샘플:")
for i in range(3):
    print(f"원본: {X_train.iloc[i]}")
    print(f"전처리: {X_train_processed[i]}")
    print()

데이터 분할 전략:
- Train: original + augment + newly_generated (모든 타입)
- Validation: original만 (원본 데이터만으로 검증)

전체 데이터: 366,943개
  - Original: 136,453개
  - Augment + Newly_generated: 230,490개
훈련 데이터: 360,120개
검증 데이터: 6,823개
훈련 데이터에 대한 전처리 파이프라인 학습 및 적용...
✓ 전처리 파이프라인 학습 완료
검증 데이터에 전처리 파이프라인 적용...

Train: 360120, Val: 6823

클래스 분포 검증:
전체 데이터:
  클래스 0: 135464개 (36.9%)
  클래스 1: 49723개 (13.6%)
  클래스 2: 120395개 (32.8%)
  클래스 3: 61361개 (16.7%)

Original 데이터:
  클래스 0: 55837개 (40.9%)
  클래스 1: 13310개 (9.8%)
  클래스 2: 48188개 (35.3%)
  클래스 3: 19118개 (14.0%)

훈련 데이터:
  클래스 0: 132672개 (36.8%)
  클래스 1: 49057개 (13.6%)
  클래스 2: 117986개 (32.8%)
  클래스 3: 60405개 (16.8%)

검증 데이터:
  클래스 0: 2792개 (40.9%)
  클래스 1: 666개 (9.8%)
  클래스 2: 2409개 (35.3%)
  클래스 3: 956개 (14.0%)

타입별 분포:
훈련 데이터 타입 분포:
  augment: 139,818개 (38.8%)
  original: 129,630개 (36.0%)
  newly_generated: 90,672개 (25.2%)

검증 데이터 타입 분포:
  original: 6,823개 (100.0%)

전처리 결과 샘플:
원본: 상류층여대생얘기ㅋㅋㅋㅋ
전처리: 상류층여대생얘기ㅋㅋㅋ

원본: it's good man.
전처리: it s good man.

원본: 뭐

# 데이터 저장

전처리된 데이터를 CSV 파일로 저장하여 모델 학습 노트북에서 사용할 수 있도록 합니다.


In [15]:
# 전처리된 데이터를 CSV 파일로 저장
import os

# data 디렉토리가 없으면 생성
os.makedirs("data", exist_ok=True)

TRAIN_DATA_FILE = "data/train_final_newly_gen_added.csv"
VAL_DATA_FILE = "data/val_final.csv"

# 훈련 데이터 저장
train_data.to_csv(TRAIN_DATA_FILE, index=False)
print(f"✅ 훈련 데이터 저장 완료: {TRAIN_DATA_FILE}({len(train_data):,}개)")

# 검증 데이터 저장
val_data.to_csv(VAL_DATA_FILE, index=False)
print(f"✅ 검증 데이터 저장 완료: {VAL_DATA_FILE} ({len(val_data):,}개)")

# 전처리 파이프라인 정보 저장 (선택사항)
preprocessing_info = {
    'total_original_samples': len(df),
    'total_clean_samples': len(df_clean),
    'removed_samples': removed_info['total_removed'],
    'train_samples': len(train_data),
    'val_samples': len(val_data),
    'preprocessing_pipeline_fitted': preprocessor.is_fitted
}

print(f"\n=== 전처리 완료 요약 ===")
print(f"원본 데이터: {preprocessing_info['total_original_samples']:,}개")
print(f"품질 검사 후: {preprocessing_info['total_clean_samples']:,}개")
print(f"제거된 데이터: {preprocessing_info['removed_samples']:,}개")
print(f"훈련 데이터: {preprocessing_info['train_samples']:,}개")
print(f"검증 데이터: {preprocessing_info['val_samples']:,}개")
print(f"전처리 파이프라인 학습 완료: {preprocessing_info['preprocessing_pipeline_fitted']}")

print(f"\n✅ 전처리 완료! 다음 단계: train_pytorch.ipynb에서 모델 학습을 진행하세요.")


✅ 훈련 데이터 저장 완료: data/train_final_newly_gen_added.csv(360,120개)
✅ 검증 데이터 저장 완료: data/val_final.csv (6,823개)

=== 전처리 완료 요약 ===
원본 데이터: 366,943개
품질 검사 후: 366,943개
제거된 데이터: 7,701개
훈련 데이터: 360,120개
검증 데이터: 6,823개
전처리 파이프라인 학습 완료: True

✅ 전처리 완료! 다음 단계: train_pytorch.ipynb에서 모델 학습을 진행하세요.


In [16]:
test_data = pd.read_csv("data/test.csv")

test_data.head()

test_data.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59928 entries, 0 to 59927
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      59928 non-null  int64 
 1   review  59927 non-null  object
dtypes: int64(1), object(1)
memory usage: 936.5+ KB


In [17]:
processed_test_data = preprocessor.transform(test_data["review"].tolist())

processed_test_data

test_data["review"] = processed_test_data

test_data.head()

test_data.to_csv("data/test_processed.csv", index=False)





In [1]:
train_data.head()

NameError: name 'train_data' is not defined